<div style="text-align: center;"><br>
<img src="https://assets-global.website-files.com/62b9d45fb3f64842a96c9686/62d84db4aeb2f6552f3a2f78_Quantinuum%20Logo__horizontal%20blue.svg" width="200" height="200" /></div>

# Circuit Compilation

Circuits submitted to Quantinuum H-Series quantum computers and emulators are automatically run through TKET compilation passes for H-Series hardware. This enables circuits to be automatically optimized for H-Series systems and run more efficiently.

When using `pytket` before submitting to hardware, the `get_compiled_circuit` function performs the same compilation passes run after submission to Quantinuum systems. Local compilation enables end users to visualize circuit before submission to hardware, and to determine if a different optimization level is desired.  **The default compilation setting for circuits submitted to H-Series sytems is optimization level 2.** 

The circuit to be submitted to hardware must satisfy a gate set predicate. This check is performed internally within `pytket-quantinuum`. The `get_compiled_circuit` (`get_compiled_circuits`) instance method rewrites the circuit for submission in the H-Series native gate set.

More information on the specific compilation passes applied can be found on the `pytket-quantinuum` documentation, specifically the [Default Compilation](https://tket.quantinuum.com/extensions/pytket-quantinuum/#default-compilation) section.

### Workflow

Set up the quantum circuit.

In [ ]:
from pytket.circuit import Circuit
from pytket.circuit.display import render_circuit_jupyter
from pytket.extensions.quantinuum import QuantinuumBackend

In [ ]:
circuit = Circuit(2, name="Bell State")
circuit.H(0)
circuit.CX(0, 1)
circuit.measure_all()
machine = "H1-1E"
backend = QuantinuumBackend(device_name=machine)
backend.login()

The example below corresponds to a case where the user wants optimization level 2. The user can look at the optimizations applied before submitting the circuit. 

In [ ]:
n_shots = 100
compiled_circuit = backend.get_compiled_circuit(circuit, optimisation_level=2)

render_circuit_jupyter(compiled_circuit)

In [ ]:
handle = backend.process_circuit(compiled_circuit, n_shots=n_shots)

<div align="center"> &copy; 2024 by Quantinuum. All Rights Reserved. </div>